<a href="https://colab.research.google.com/github/ernest1027/dsc-project/blob/main/BADDIEProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pip install --upgrade --force-reinstall `cat /content/gdrive/My\ Drive/BADDIE_installed.txt`

ERROR: Invalid requirement: '@'


In [ ]:
import torch
import zipfile
import torchaudio
from glob import glob

In [ ]:
torch.cuda.is_available()

True

In [ ]:
#speech to text

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')  

model, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                       model='silero_stt',
                                       language='en', 
                                       device=device)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils  


test_files = glob('ted_talk.wav')
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]),
                            device=device)

output = model(input)


Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master


  0%|          | 0.00/112M [00:00<?, ?B/s]

IndexError: ignored

In [ ]:
#repunctuation

model, example_texts, languages, punct, apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                  model='silero_te')
output_text = ''
for example in output:
  print(decoder(example.cpu()));
  input_text = decoder(example.cpu())
  print(apply_te(input_text, lan='en'))
  output_text += apply_te(input_text, lan='en')
print(output_text)

Downloading: "https://github.com/snakers4/silero-models/archive/master.zip" to /root/.cache/torch/hub/master.zip


  0%|          | 0.00/87.5M [00:00<?, ?B/s]

NameError: ignored

In [ ]:
# Best model that we can use for abstractive summarization, pre-trained with many quality datasets

# Need to work on beam length to ensure that the summarized text is not only a sentence long
from transformers import PegasusForConditionalGeneration, PegasusTokenizer


class AbstractiveSummarization:
  def __init__(self, input_text):
    self.input_text = input_text

  def Summarize(self):    
    model_name = "google/pegasus-xsum"
    pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
    tokenizer = PegasusTokenizer.from_pretrained(model_name)
    tokens = tokenizer(self.input_text, batched=True, truncation=True, padding='longest', return_tensors='pt').to(device)
    # new set of token sequences generated
    summary = pegasus_model.generate(**tokens)
    decoded_summary = tokenizer.decode(summary[0])

    return decoded_summary

In [ ]:
from transformers import Trainer, TrainingArguments, PegasusTokenizer, DataCollatorWithPadding
from datasets import load_dataset

# probably using CSV?
# Need to create dataset based off of lecture files and find a way to summarize
class PegasusTraining:
  def __init__(self, dataset_path):
    self.dataset_name=dataset_path

  def preprocess(data):
      return tokenizer(data["text"], truncation=True)
  
  def TrainModel:
    pegasus_training_model=PegasusForConditionalGeneration.from_pretrained(model_name, num_labels=2).to(device)

    summarization_dataset = load_dataset("csv", data_files=self.datasetpath, sep=";")
    summarization_dataset["train"][0]
    tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
    tokenized_dataset = summarization_dataset.map(preprocess, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


    training_args = TrainingArguments(
        output_dir="./results",
        learning_rate=2e-5,
        per_device_train_batch_size=16,     # need to clarify args to fit our needs
        per_device_eval_batch_size=16,      # Also need to setup instantiation
        num_train_epochs=5,
        weight_decay=0.01,
    )

    trainer = Trainer(
        model=pegasus_training_model,
        args=training_args,
        train_dataset=tokenized_
        # training dataset
        ["train"],
        eval_dataset=tokenized_
        # training dataset
        ["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    trainer.train()

NameError: ignored

In [ ]:
!nvidia-smi


Sat Mar 12 05:16:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------